In [1]:
import os

import agenticblocks as ab
import dotenv

In [2]:
dotenv.load_dotenv("../.env")

True

In [3]:
model = ab.Model(
    model_name="openai/gpt-4o",
    system_prompt="You are a helpful assistant.",
    api_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

In [6]:
# Local model example (Ollama)
local_model = ab.LocalModel("gemma3", provider="ollama")
local_model("How many r's are in strawberry?")


'There are three "r"s in the word "strawberry". \n\nLet me know if you\'d like to try another word puzzle!'

In [4]:
class CoT:
    def __init__(self, model):
        self.model = model
        self.template = "{prompt}\nLet's think step by step." 

    def __call__(self, prompt, **kwargs):
        return self.model(self.template.format(prompt=prompt), **kwargs)

cot_block = CoT(model)
cot_block("How many r's are in strawberry?")

{'content': 'To find out how many \'r\'s are in the word "strawberry," let\'s break the word down letter by letter:\n\n1. s\n2. t\n3. r\n4. a\n5. w\n6. b\n7. e\n8. r\n9. r\n10. y\n\nNow, let\'s identify the \'r\'s:\n\n- The first \'r\' is the 3rd letter.\n- The second \'r\' is the 8th letter.\n- The third \'r\' is the 9th letter.\n\nThere are three \'r\'s in the word "strawberry."',
 'extra': {'response': {'id': 'gen-1767443851-P3ZyvrBVJ5aX0G9AVLeM',
   'choices': [{'finish_reason': 'stop',
     'index': 0,
     'logprobs': None,
     'message': {'content': 'To find out how many \'r\'s are in the word "strawberry," let\'s break the word down letter by letter:\n\n1. s\n2. t\n3. r\n4. a\n5. w\n6. b\n7. e\n8. r\n9. r\n10. y\n\nNow, let\'s identify the \'r\'s:\n\n- The first \'r\' is the 3rd letter.\n- The second \'r\' is the 8th letter.\n- The third \'r\' is the 9th letter.\n\nThere are three \'r\'s in the word "strawberry."',
      'refusal': None,
      'role': 'assistant',
      'reaso

In [5]:
model.messages

[{'role': 'system',
  'content': 'You are a helpful assistant.',
  'timestamp': 1767443849.957748}]

In [6]:
class CoTSC:
    def __init__(self, cot_block, N=5, temperature=0.7):
        self.cot_block = cot_block
        self.N = N
        self.temperature = temperature
        self.aggregator = ab.Model(
            "openai/gpt-4o",
            api_url="https://openrouter.ai/api/v1",
            api_key=os.getenv("OPENROUTER_API_KEY"),
        )
        self.template = "{responses}\nGiven the responses above. Output the most common answer."
    
    def __call__(self, prompt):
        responses = []
        for _ in range(self.N):
            responses += [self.cot_block(prompt, temperature=self.temperature)["content"]]
        responses = "\n\n".join(responses)
        return self.aggregator(self.template.format(responses=responses))

cotsc_block = CoTSC(cot_block)
cotsc_block("How many r's are in strawberry?")

{'content': 'There are a total of 3 \'r\'s in the word "strawberry."',
 'extra': {'response': {'id': 'gen-1767443865-qtwQ5VflUzrfvz8lHczt',
   'choices': [{'finish_reason': 'stop',
     'index': 0,
     'logprobs': None,
     'message': {'content': 'There are a total of 3 \'r\'s in the word "strawberry."',
      'refusal': None,
      'role': 'assistant',
      'reasoning': None},
     'native_finish_reason': 'stop'}],
   'created': 1767443865,
   'model': 'openai/gpt-4o',
   'object': 'chat.completion',
   'system_fingerprint': 'fp_deacdd5f6f',
   'usage': {'completion_tokens': 18,
    'prompt_tokens': 798,
    'total_tokens': 816,
    'completion_tokens_details': {'reasoning_tokens': 0, 'image_tokens': 0},
    'prompt_tokens_details': {'audio_tokens': 0,
     'cached_tokens': 0,
     'video_tokens': 0},
    'cost': 0.002175,
    'is_byok': False,
    'cost_details': {'upstream_inference_cost': None,
     'upstream_inference_prompt_cost': 0.001995,
     'upstream_inference_completions

In [7]:
model.messages

[{'role': 'system',
  'content': 'You are a helpful assistant.',
  'timestamp': 1767443849.957748}]

In [8]:
cotsc_block.aggregator.messages

[]

In [1]:
import agenticblocks as ab

In [7]:
cot = ab.ChainOfThought(
    ab.Model(
        "openai/gpt-4o",
        api_url="https://openrouter.ai/api/v1",
        api_key=os.getenv("OPENROUTER_API_KEY"),
    ),
)

In [9]:
sr = ab.SelfRefine(
    ab.Model(
        "openai/gpt-4o",
        api_url="https://openrouter.ai/api/v1",
        api_key=os.getenv("OPENROUTER_API_KEY"),
    ),
)

In [8]:
cot("How many r's are in strawberry?")

{'content': 'To determine how many "r\'s" are in the word "strawberry," let\'s examine the word step by step:\n\n1. **Identify the Word:** The word is "strawberry."\n2. **Spell Out Each Letter:** \n   - S\n   - T\n   - R\n   - A\n   - W\n   - B\n   - E\n   - R\n   - R\n   - Y\n\n3. **Count the "R\'s":**\n   - First "R" is the 3rd letter.\n   - Second "R" is the 8th letter.\n   - Third "R" is the 9th letter.\n\nSo, there are 3 "r\'s" in the word "strawberry."',
 'extra': {'response': {'id': 'gen-1767447666-LXYJ5cgqZnfPpFb33V27',
   'choices': [{'finish_reason': 'stop',
     'index': 0,
     'logprobs': None,
     'message': {'content': 'To determine how many "r\'s" are in the word "strawberry," let\'s examine the word step by step:\n\n1. **Identify the Word:** The word is "strawberry."\n2. **Spell Out Each Letter:** \n   - S\n   - T\n   - R\n   - A\n   - W\n   - B\n   - E\n   - R\n   - R\n   - Y\n\n3. **Count the "R\'s":**\n   - First "R" is the 3rd letter.\n   - Second "R" is the 8th l

In [10]:
sr("How many r's are in strawberry?")

{'content': 'The word "strawberry" consists of the letters s, t, r, a, w, b, e, r, r, y. Upon evaluating each letter, the letter \'r\' appears twice, confirming that there are two \'r\'s in "strawberry." As a fun fact, the name "strawberry" may originate from the practice of mulching the plants with straw to protect them from pests or from the plant\'s tendency to "strew" or spread out runners. This adds an interesting layer of history to the word while addressing the task at hand.',
 'extra': {'history': [{'response': 'The word "strawberry" contains three \'r\'s.',
    'critique': None},
   {'response': 'Corrected Response:\nThe word "strawberry" contains two \'r\'s.',
    'critique': 'The response incorrectly identifies the number of \'r\'s in the word "strawberry." There are actually two \'r\'s in "strawberry," not three. This mistake undermines the accuracy and reliability of the response.\n\nTo improve the response, it should provide the correct count of \'r\'s:\n\nCorrected Respo